## Notebook for plotting gene specific eQTL results

#### import libraries and set notebook variables

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# parameters
cohort = 'foundin'
gene = 'lnc-RHEX-8'

In [ ]:
# naming
# cohort_build = f'{cohort}.{day}'
geno_version = 'amppdv1'
cohort_version = f'{cohort}.{geno_version}'
days = ['da0', 'da25', 'da65']

# directories
wrk_dir = f'/home/jupyter/{cohort}/eqtl'
expr_dir = f'{wrk_dir}/expression'
genos_dir = f'{wrk_dir}/genotypes'
info_dir = f'{wrk_dir}/sample_info'
tensorqtl_dir = f'{wrk_dir}/tensorqtl'
gwas_dir = f'{wrk_dir}/gwas'

# input files
gencode_pkl = f'{expr_dir}/gencode_v29.lncipedia_v5_2_hc.annotation.pkl'

# output files
# eqtl_psam = f'{info_dir}/{cohort_build}.psam'

# constant values
autosomes = [str(x) for x in list(range(1,23))]
max_dist = 1000000
max_threads = 24
alpha_value = 0.05

# items of interest
meta5_stats_file = f'{gwas_dir}/pdmeta_sumstats_hg38.h5'

#### load the gencode annotations

In [ ]:
%%time
gencode_df = pd.read_pickle(gencode_pkl)
print(gencode_df.shape)

#### load the full gwas summary stats

In [ ]:
%%time
gwas_stats_df = pd.read_hdf(meta5_stats_file)
print(gwas_stats_df.shape)
display(gwas_stats_df.sample(5))

#### load the cis-eQTL results by day for the specified gene

In [ ]:
%%time

gene_df = gencode_df.loc[gencode_df['gene_name'] == gene]
#     print(gene_df['seqname'].unique()[0])
gene_id =gene_df['gene_id'].unique()[0]
chrom = gene_df['seqname'].unique()[0]
gene_name = gene_df['gene_name'].unique()[0]
gene_start = gene_df['start'].min()
gene_stop = gene_df['end'].max()

gene_eqtl_df = None
for day in days:
    # now load the chromosome eqtl results and extract specific gene results
    chrom_eqtl_df = pd.read_parquet(f'{tensorqtl_dir}/{cohort}.{day}.cis_qtl_pairs.{chrom}.parquet')
    chrom_bim_df = pd.read_csv(f'{genos_dir}/{cohort_version}.{chrom}.bfile.bim', 
                               header=None, sep='\s+')
    chrom_bim_df.columns = ['chr', 'name', 'cm', 'pos', 'a1', 'a2']
    chrom_eqtl_df = chrom_eqtl_df.merge(chrom_bim_df, how='inner', 
                                    left_on='variant_id', right_on='name')    
#     print(chrom_eqtl_df.shape)
    this_df = chrom_eqtl_df.loc[chrom_eqtl_df['phenotype_id'] == gene_id].copy()
    this_df['set'] = day
    gene_eqtl_df = pd.concat([gene_eqtl_df, this_df])
#     print(gene_eqtl_df.shape)

#### get gwas stats for region of specified gene

In [ ]:
region_gwas_df = gwas_stats_df.loc[gwas_stats_df['SNP'].isin(gene_eqtl_df['variant_id'])].copy()
region_gwas_df['set'] = 'PDrisk'
print(region_gwas_df.shape)

#### subset appropriate columns and rename so that eqtl and gwas can be concat'd

In [ ]:
eqtl_columns = ['variant_id', 'chr', 'pos', 'slope', 'slope_se', 'pval_nominal','set']
gene_eqtl_df = gene_eqtl_df[eqtl_columns]
gene_eqtl_df.columns = ['variant', 'chr', 'pos', 'beta', 'se', 'pvalue','set']
print(gene_eqtl_df.shape)
# display(gene_eqtl_df.head())

gwas_columns = ['SNP', 'chr', 'position', 'b', 'se', 'p','set']
region_gwas_df = region_gwas_df[gwas_columns]
region_gwas_df.columns = ['variant', 'chr', 'pos', 'beta', 'se', 'pvalue','set']
print(region_gwas_df.shape)
# display(region_gwas_df.head())

results_df = pd.concat([gene_eqtl_df, region_gwas_df])
results_df['log10_pvalue'] = np.log10(results_df['pvalue'])*-1
results_df['z_score'] = results_df['beta']/results_df['se']
results_df['z_score_abs'] = np.abs(results_df['z_score'])
print(results_df.shape)
display(results_df.sample(5))

#### now do the plotting

In [ ]:
#plot local manhattan for gene eQTL
def plot_eqtl_manhattan(gene_id, gene_name, gene_chrom, gene_start, gene_stop, 
                        results_df):
    print(f'{gene_name} {gene_id}')
    print(f'gene {gene_name} is on {gene_chrom} from {gene_start} to {gene_stop}')

    #pull in all results for the gene from chromosome for all visits
    print(results_df.shape)

    #now actually do the plotting
    sns.set(style='darkgrid')
    sns.relplot(x='pos',y='log10_pvalue',size='z_score_abs', hue='set',
                alpha=.5, palette="dark", height=12, data=results_df)        

    min_y = round(min(results_df['log10_pvalue']))

    plt.plot([gene_start, gene_stop], [min_y, min_y], linewidth=5, color='black')
    plt.text(gene_stop+10000, min_y, gene_name, fontsize='large', color='black')

    plt.title(f'{gene_name} eQTL', fontsize='large') 
    plt.xlabel(f'Bp on chromosome {gene_chrom}')
    plt.ylabel(f'-log10(p-value)')    
    plt.show()

#   plot_out_file_name = f'{WRKDIR}/plink/images/{gene_name}.local_man.png'
#   plt.savefig(plot_out_file_name,format='png',dpi=600,bbox_inches='tight')
    
    return

In [ ]:
plot_eqtl_manhattan(gene_id, gene_name, chrom, gene_start, gene_stop, 
                    results_df)

In [ ]:
gene_eqtl_df.columns = ['eqtl_'+ x for x in gene_eqtl_df.columns]
region_gwas_df.columns = ['gwas_'+ x for x in region_gwas_df.columns]
display(gene_eqtl_df.head())
display(region_gwas_df.head())


In [ ]:
merged_df = gene_eqtl_df.merge(region_gwas_df, how='inner', 
                               left_on='eqtl_variant', right_on='gwas_variant')
print(merged_df.shape)
display(merged_df.head())

In [ ]:
# sns.relplot(x='gwas_beta',y='eqtl_beta', hue='eqtl_set', size=np.abs(merged_df['gwas_beta']),
#             alpha=.5, palette="dark", height=12, data=merged_df) 

In [ ]:
#plot local manhattan for gene eQTL
def plot_gwas_eqtl(gene_id, gene_name, gene_chrom, gene_start, gene_stop, 
                   this_df):
    print(f'{gene_name} {gene_id}')
    print(f'gene {gene_name} is on {gene_chrom} from {gene_start} to {gene_stop}')

    #pull in all results for the gene from chromosome for all visits
    print(this_df.shape)

    #now actually do the plotting
    sns.set(style='darkgrid')
    sns.relplot(x=np.log10(this_df['gwas_pvalue'])*-1, 
                y=np.log10(this_df['eqtl_pvalue'])*-1, 
                size=np.abs(this_df['gwas_beta']), 
                hue='eqtl_set', alpha=.5, palette="dark", 
                height=12, data=this_df)         
    
    plt.title(f'PD risk and {gene_name} eQTL', fontsize='large')
    plt.xlabel('PD GWAS risk -log10(p-value)')
    plt.ylabel(f'{gene_name} eQTL -log10(p-value)')
    plt.show()

#   plot_out_file_name = f'{WRKDIR}/plink/images/{gene_name}.local_man.png'
#   plt.savefig(plot_out_file_name,format='png',dpi=600,bbox_inches='tight')
    
    return

In [ ]:
plot_gwas_eqtl(gene_id, gene_name, chrom, gene_start, gene_stop, merged_df)